In [1]:
import sys
import os

# Add the parent directory of 'src' to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

In [2]:
from sqlalchemy import create_engine, select, Column, Integer, String, Boolean, DateTime
from sqlalchemy.orm import Session, DeclarativeBase, Mapped, mapped_column
from datetime import datetime

class Base(DeclarativeBase):
    pass

class Todo(Base):
    __tablename__ = 'Todo'
    
    id: Mapped[int] = mapped_column(Integer, primary_key=True)
    title: Mapped[str] = mapped_column(String, nullable=False)
    completed: Mapped[bool] = mapped_column(Boolean, default=False)
    createdAt: Mapped[int] = mapped_column(Integer)

    @property
    def created_at(self):
        return datetime.fromtimestamp(self.createdAt / 1000)

    def __repr__(self):
        return f"<Todo(id={self.id}, title='{self.title}', completed={self.completed}, createdAt={self.created_at})>"

# Create the engine
engine = create_engine('sqlite:///../prisma/dev.db')


# with engine.connect() as connection:
#     result = connection.execute(select(Todo.__table__))
#     for row in result:
#         print(row)

with Session(engine) as session:
    stmt = select(Todo)
    todos = session.execute(stmt).scalars().all()

    for todo in todos:
        print(f"ID: {todo.id}, Title: {todo.title}, Completed: {todo.completed}, Created At: {todo.created_at}")
        print(f"Raw timestamp: {todo.createdAt}")

ID: 1, Title: Say hello, Completed: False, Created At: 2024-10-20 21:35:34.593000
Raw timestamp: 1729474534593


In [6]:
from typing import List
from prisma import Prisma
from src.models.todo import Todo

async def get_todos() -> List[Todo]:
    """
    Retrieve all todos from the database synchronously.

    This function fetches all todo items stored in the database and returns them as a list of Todo objects.

    Returns:
        List[Todo]: A list of Todo objects representing all the todos in the database.

    Raises:
        Exception: If there's an error while fetching the todos from the database.
    """
    try:
        db = Prisma()
        await db.connect()
        todos = await db.todo.find_many()
        return [
            Todo(
                id=todo.id,
                title=todo.title,
                completed=todo.completed,
                createdAt=todo.createdAt,
            )
            for todo in todos
        ]
    except Exception as e:
        raise Exception(f"Failed to fetch todos: {str(e)}")
    finally:
        await db.disconnect()



todos = await get_todos()
print(todos)

[Todo(id=1, title='Say hello', completed=False, createdAt=datetime.datetime(2024, 10, 21, 1, 35, 34, 593000, tzinfo=TzInfo(UTC)))]


/var/folders/qj/sdy1tgv53cg26v08n8n8g55w0000gn/T/ipykernel_32297/1756116166.py:37: RuntimeWarning: coroutine 'get_todos' was never awaited
  todos = await get_todos()
